Notebook created for the sole porpuse to explore psycopg2 in order to make querys and make the pipeline to create the table.

In [1]:
import psycopg2
from psycopg2 import sql
import pandas as pd

# Configuración de conexión
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'mydatabase',
    'user': 'NicolasSeivane',
    'password': 'Metlife2025'
}

try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Conexión exitosa a la base de datos PostgreSQL.")
except Exception as e:
        print(f"❌ Error al conectar a la base de datos: {e}")
        print("\n💡 Asegúrate de que Docker esté corriendo y los contenedores estén activos:")
        print("   docker-compose up -d")

❌ Error al conectar a la base de datos: 

💡 Asegúrate de que Docker esté corriendo y los contenedores estén activos:
   docker-compose up -d


In [ ]:
datos = pd.read_csv("../data/raw/dataset.csv")


for c, t in zip(datos.columns, datos.dtypes):
    print(c, t)
    print(str(t))


def create_table_query(df, table_name):
    cols = []
    for c, t in zip(df.columns, df.dtypes):
        if "int" in str(t):
            cols.append(f"{c} INTEGER")
        elif "float" in str(t):
            cols.append(f"{c} FLOAT")
        elif "bool" in str(t):
            cols.append(f"{c} BOOLEAN")
        elif "datetime" in str(t):
            cols.append(f"{c} TIMESTAMP")
        elif "object" in str(t):
            cols.append(f"{c} TEXT")
        else:
            print(f"Tipo de dato no manejado para la columna {c} con tipo {t}, se asigna TEXT por defecto.")
            cols.append(f"{c} TEXT")
        
    cols_sql = ", ".join(cols)
    return f"CREATE TABLE IF NOT EXISTS {table_name} ({cols_sql});"

cursos = conn.cursor()
cursos.execute(create_table_query(datos, "training_dataset "))
conn.commit()
cursos.close()

age int64
int64
sex object
object
bmi float64
float64
children int64
int64
smoker object
object
region object
object
charges float64
float64


'CREATE TABLE IF NOT EXISTS training_dataset  (age INTEGER, sex TEXT, bmi FLOAT, children INTEGER, smoker TEXT, region TEXT, charges FLOAT);'

In [8]:
def insertar_datos_desde_df(conn, df, table_name):
    cursor = conn.cursor()
    for index, row in df.iterrows(): ## Separa el index y los row (Series) de cada fila para hacer la query.
        placeholders = ', '.join(['%s'] * len(row)) # Crea una cadena con tantos '%s' como columnas tiene la fila, separados por coma.
        insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        cursor.execute(insert_query, tuple(row)) ## Aca reemplaza los '%s' por los valores de la fila.
    conn.commit()
    cursor.close()
insertar_datos_desde_df(conn, datos, "training_dataset ")

NameError: name 'conn' is not defined

In [9]:
for index, row in datos.iterrows():
        columns = ', '.join(datos.columns)
        values = [row[c] for c in datos.columns]
        insert_query = f"INSERT INTO training_dataset ({columns}) VALUES ({values})"
print(insert_query)

INSERT INTO training_dataset (age, sex, bmi, children, smoker, region, charges) VALUES ([61, 'female', 29.07, 0, 'yes', 'northwest', 29141.3603])
